In [ ]:
import sys
from pathlib import Path

In [ ]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
sys.path.append(str(Path("../../ssl").resolve()))

In [ ]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.meta_pseudo_label.meta_pseudo_label import MetaPseudoLabelTrainer
from src.trainers.meta_pseudo_label.meta_pseudo_label_config import MetaPseudoLabelTrainerConfig
from src.data_loaders.meta_pseudo_label.meta_pseudo_label_config import MetaPseudoLabelDataLoaderConfig
from src.data_loaders.meta_pseudo_label.meta_pseudo_label import MetaPseudoLabelDataLoader

# Description

In this notebook, the AlexNet model will be trained on the CIFAR-10 dataset using only 25% of the labelled dataset.

## Set up Experiment

In [ ]:
class TrainerConfig(MetaPseudoLabelTrainerConfig):
    num_epochs = 200
    loss_ramp_up_epochs = 150
    unsup_loss_weight = 3.0
    alpha = 0.99

train_config = TrainerConfig()

In [ ]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [ ]:
class DataLoaderConfig(MetaPseudoLabelDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 50000 # dataset size    

    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config = DataLoaderConfig()

## Get Datasets

In [ ]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# get train dataset by subsampling 10% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.10, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [ ]:
x_train = np.concatenate((x_train_unlabelled, x_train_labelled), axis = 0)
y_train = np.concatenate((y_train_unlabelled, y_train_labelled), axis = 0)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

In [ ]:
# create train dataset
train_data = MetaPseudoLabelDataLoader(train_data, data_loader_config)(training = True)

In [ ]:
# create test dataset
val_data = MetaPseudoLabelDataLoader(val_data, data_loader_config)(training = False)

In [ ]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

# Train Model

In [ ]:
model_student = PiModel(model_config)()
model_teacher = PiModel(model_config)()

In [ ]:
trainer = MetaPseudoLabelTrainer(
    model_student,
    model_teacher,
    train_data,
    train_config,
    val_dataset = val_data)

In [ ]:
trainer.train()